In [ ]:
import pandas as pd
from pandas_datareader import DataReader as pdr
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px

# U.S. STOCK MARKET RETURN

## VOLATILITY

In [ ]:
mkt = pdr('F-F_Research_Data_Factors_Daily','famafrench',start=1926)[0]/100
mkt = mkt['Mkt-RF'] + mkt.RF
mkt_vol = mkt.resample("M").apply(lambda x: x.std())[:-1]
mkt_vol.index = mkt_vol.index.to_period("M").astype(str)
mkt_vol.name = "market volatility"
mkt_vol.tail()


## MONTHLY RETURNS

In [ ]:
mkt = pdr('F-F_Research_Data_Factors','famafrench',start=1926)[0]/100
mkt_ret = mkt['Mkt-RF'] + mkt.RF
mkt_ret.index = mkt_ret.index.astype(str)
mkt_ret.name = "market return"
mkt_ret.tail()

# RETURNS FROM YAHOO FINANCE

In [ ]:
ticker = input("Input a ticker")
price = yf.download(ticker, start="1970-01-01")["Adj Close"]

## VOLATILITY

In [ ]:
ret_daily = price.pct_change()
ret_vol = ret_daily.resample("M").apply(lambda x: x.std())[:-1]
ret_vol.index = ret_vol.index.to_period("M").astype(str)
ret_vol.name = ticker + " volatility"
ret_vol.tail()

## MONTHLY RETURNS

In [ ]:
price_monthly = price.resample("M").last()
ret_ret = price_monthly.pct_change()[:-1]
ret_ret.index = ret_ret.index.to_period("M").astype(str)
ret_ret.index.name = "month"
ret_ret.name = ticker + " return"
ret_ret.tail()

# SELECT YAHOO RETURN OR MARKET

In [ ]:
flag = int(input("input 0 for market or 1 for yahoo return"))
data = "ret" if flag else "mkt"
name = ticker.upper() if flag else "Market" 
vol = eval(data + "_vol")
ret = eval(data + "_ret")
df = pd.concat((vol, ret), axis=1).dropna()
df.columns = ["vol", "ret"]

# FIGURE 1

In [ ]:
trace = go.Scatter(
    x=vol.index.to_list(), 
    y=vol, 
    mode="lines", 
    hovertemplate="%{x}<br>%{y:.1%}<extra></extra>"
)
fig = go.Figure(trace)
fig.update_layout(
    yaxis_tickformat=".0%",
    yaxis_title = f"{name} Volatility",
    template="plotly_white",
)
fig.show()

# FIGURE 2

In [ ]:
fig = px.scatter(
    x=vol[:-1],
    y=vol[1:],
    trendline="ols",
)
fig.update_layout(
    xaxis_title=f"Lagged {name} Volatility",
    yaxis_title=f"{name} Volatility",
    xaxis_tickformat=".0%",
    yaxis_tickformat=".0%",
    template="plotly_white",
)
fig.update_traces(
    marker=dict(
        size=12, 
        line=dict(width=2, color="DarkSlateGrey")
    ),
    selector=dict(mode="markers"),
    text=df.index.to_list()[1:],
    hovertemplate="%{text}<br>lagged vol = %{x}<br>vol = %{y}<extra></extra>"
)
fig.show()


# FIGURE 3

In [ ]:
fig = px.scatter(
    x=df.vol[:-1],
    y=df.ret[1:],
    trendline="ols",
)
fig.update_layout(
    xaxis_title=f"Lagged {name} Volatility",
    yaxis_title=f"{name} Return",
    xaxis_tickformat=".0%",
    yaxis_tickformat=".0%",
    template="plotly_white",
)
fig.update_traces(
    marker=dict(
        size=12, 
        line=dict(width=2, color="DarkSlateGrey")
    ),
    selector=dict(mode="markers"),
    text=df.index.to_list()[1:],
    hovertemplate="%{text}<br>lagged vol = %{x}<br>return = %{y}<extra></extra>"
)
fig.show()

# FIGURE 4

In [ ]:
fig = px.scatter(
    x=df.vol,
    y=df.ret,
    trendline="ols",
)
fig.update_layout(
    xaxis_title=f"{name} Volatility",
    yaxis_title=f"{name} Return",
    xaxis_tickformat=".0%",
    yaxis_tickformat=".0%",
    template="plotly_white",
)
fig.update_traces(
    marker=dict(
        size=12, 
        line=dict(width=2, color="DarkSlateGrey")
    ),
    selector=dict(mode="markers"),
    text=df.index.to_list(),
    hovertemplate="%{text}<br>vol = %{x}<br>return = %{y}<extra></extra>"
)
fig.show()